In [187]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from numpy import exp
import json

In [188]:
# Achieves over 90% accuracy on MNIST
data = pd.read_csv("/Users/aryank/Developer/NNs/neural-network-from-scratch/train.csv")
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [189]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

In [190]:
# Transpose to make every column an example, instead of row
dev = data[0:1000].T 
Y_dev = dev[0]
X_dev = dev[1:n]
X_dev = X_dev / 255.0 

train = data[1000:m].T
Y_train = train[0]
X_train = train[1:n]
X_train = X_train / 255.0

In [191]:
# First example
X_train[:, 0].shape

(784,)

In [192]:
# Parameters
def init_params():
    W1 = np.random.randn(64, 784) * np.sqrt(2 / 784)
    b1 = np.zeros((64, 1))
    W2 = np.random.randn(10, 64) * np.sqrt(2 / 64)
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return exp(Z - np.max(Z, axis=0, keepdims=True)) / np.sum(exp(Z - np.max(Z, axis=0, keepdims=True)), axis=0, keepdims=True)

def forward(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

    

In [193]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def derivative_ReLU(Z):
    return Z > 0

def backprop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2

In [194]:
def update_wandb(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [195]:
def predictions(A2):
    return np.argmax(A2, 0)

def accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backprop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_wandb(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 10 == 0:
            print(f"i: {i}, Acc: {accuracy(predictions(A2), Y)}")

            # save the model checkpoints
            np.savez(f"model_checkpoint_{i}.npz", W1, b1, W2, b2)

    return W1, b1, W2, b2

In [196]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)

# save the model in both npz and json
np.savez('model.npz', W1, b1, W2, b2)
json.dump({'W1': W1.tolist(), 'b1': b1.tolist(), 'W2': W2.tolist(), 'b2': b2.tolist()}, open('model.json', 'w'))

i: 0, Acc: 0.15504878048780488
i: 10, Acc: 0.5897317073170731
i: 20, Acc: 0.7404146341463415
i: 30, Acc: 0.7971463414634147
i: 40, Acc: 0.8258048780487804
i: 50, Acc: 0.8424634146341463
i: 60, Acc: 0.8534146341463414
i: 70, Acc: 0.8618536585365854
i: 80, Acc: 0.8677560975609756
i: 90, Acc: 0.8724878048780488
i: 100, Acc: 0.877
i: 110, Acc: 0.8808048780487805
i: 120, Acc: 0.8840243902439024
i: 130, Acc: 0.8865121951219512
i: 140, Acc: 0.8889024390243903
i: 150, Acc: 0.891
i: 160, Acc: 0.8930243902439025
i: 170, Acc: 0.8951219512195122
i: 180, Acc: 0.8966829268292683
i: 190, Acc: 0.8980731707317073
i: 200, Acc: 0.8997073170731708
i: 210, Acc: 0.9008536585365854
i: 220, Acc: 0.9024634146341464
i: 230, Acc: 0.9036341463414634
i: 240, Acc: 0.9047317073170732
i: 250, Acc: 0.9059756097560976
i: 260, Acc: 0.9067560975609756
i: 270, Acc: 0.9072926829268293
i: 280, Acc: 0.9080731707317073
i: 290, Acc: 0.909
i: 300, Acc: 0.9097804878048781
i: 310, Acc: 0.9103658536585366
i: 320, Acc: 0.9111219512